In [1]:
import pandas as pd

In [3]:
train = pd.read_csv("dataset/train.csv")
test = pd.read_csv("dataset/test.csv")
train.sample(5)

,id,label,tweet
25247,25248,0,stay calm...stay delighted...stay curious...st...
23227,23228,0,today's #socialmedia workshop went well - make...
14207,14208,0,i am thankful for baby giggles. #thankful #pos...
19125,19126,0,i believe happy girls are the prettiest girls....
20695,20696,0,looking forward to afternoon tea tomorrow @use...


In [8]:
train.shape

(31962, 3)

In [4]:
combi = train.append(test, ignore_index=True)

/home/rcrespillo/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [6]:
combi.head(10)

,id,label,tweet
0,1,0.0,@user when a father is dysfunctional and is s...
1,2,0.0,@user @user thanks for #lyft credit i can't us...
2,3,0.0,bihday your majesty
3,4,0.0,#model i love u take with u all the time in ...
4,5,0.0,factsguide: society now #motivation
5,6,0.0,[2/2] huge fan fare and big talking before the...
6,7,0.0,@user camping tomorrow @user @user @user @use...
7,8,0.0,the next school year is the year for exams.ð...
8,9,0.0,we won!!! love the land!!! #allin #cavs #champ...
9,10,0.0,@user @user welcome here ! i'm it's so #gr...


In [7]:
combi.tail(10)

,id,label,tweet
49149,49150,NaN,loving lifeðºð¸âï¸ð #createyourfutu...
49150,49151,NaN,"black professor demonizes, proposes nazi style..."
49151,49152,NaN,learn how to think positive. #positive #ins...
49152,49153,NaN,"we love the pretty, happy and fresh you! #teen..."
49153,49154,NaN,2_damn_tuff-ruff_muff__techno_city-(ng005)-web...
49154,49155,NaN,thought factory: left-right polarisation! #tru...
49155,49156,NaN,feeling like a mermaid ð #hairflip #neverre...
49156,49157,NaN,#hillary #campaigned today in #ohio((omg)) &am...
49157,49158,NaN,"happy, at work conference: right mindset leads..."
49158,49159,NaN,"my song ""so glad"" free download! #shoegaze ..."


In [9]:
df = combi

Twits should have less than 140ch

In [10]:
df[df['tweet'].str.len() > 140].head()

,id,label,tweet
7,8,0.0,the next school year is the year for exams.ð...
335,336,0.0,watching the @user leadership embrace &amp; ki...
433,434,0.0,@user * sweets ð°ð©ð¬ð­ð«â¨!!!!!! ...
474,475,0.0,this is a shockingly and increasingly #viole...
565,566,0.0,thank youðsoo muchð pretty @user i feel ...


In [11]:
df[df['tweet'].str.len() > 140].shape

(317, 3)

There must be some f*k simbols encoded in here

In [16]:
df.tweet[335]

"watching the @user leadership embrace &amp; kiss donald's ass is #pathetic &amp;  . this is now the @user pay. a pay of racism &amp; hate. #ripgop"

In [17]:
from bs4 import BeautifulSoup
BeautifulSoup(df.tweet[335], 'lxml').get_text()

"watching the @user leadership embrace & kiss donald's ass is #pathetic &  . this is now the @user pay. a pay of racism & hate. #ripgop"

In [18]:
df['tweet'] = df['tweet'].apply(lambda x : BeautifulSoup(x, 'lxml').get_text())

In [19]:
df[df['tweet'].str.len() > 140].shape

(223, 3)

In [21]:
df.tweet[433]

'@user * sweets ð\x9f\x8d°ð\x9f\x8d©ð\x9f\x8d¬ð\x9f\x8d\xadð\x9f\x8d«â\x9c¨!!!!!! * #instagood #ì\x95\x84ì\x9d´ë\x8f\x8c #å\x81¶å\x83\x8f #idola #idol #à¹\x84à¸\xadà¸\x94à¸\xadà¸¥ #followme #selfie   #ã\x81©ã\x82\x8bã\x81\x88ã\x82\x8câ\x80¦ '

In [22]:
import re
df['tweet'] = df['tweet'].apply(lambda x : re.sub(r'@[A-Za-z0-9]+','',x))

In [23]:
df.tweet[433]

' * sweets ð\x9f\x8d°ð\x9f\x8d©ð\x9f\x8d¬ð\x9f\x8d\xadð\x9f\x8d«â\x9c¨!!!!!! * #instagood #ì\x95\x84ì\x9d´ë\x8f\x8c #å\x81¶å\x83\x8f #idola #idol #à¹\x84à¸\xadà¸\x94à¸\xadà¸¥ #followme #selfie   #ã\x81©ã\x82\x8bã\x81\x88ã\x82\x8câ\x80¦ '

In [24]:
df.shape

(49159, 3)

In [25]:
df.tweet.str.contains('https|http', na=False).count()

49159

Cleaning special simbols and numbers like # and @ -> better f-1 without this

In [27]:
df.tweet[0]

' when a father is dysfunctional and is so selfish he drags his kids into his dysfunction.   #run'

In [15]:
#df['tweet'] = df['tweet'].apply(lambda x : re.sub("[^a-zA-Z]", " ", x))

In [28]:
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer 

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/rcrespillo/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [29]:
def tweets_lemm(tweet):
    lemmatizer = WordNetLemmatizer()
    word_list = nltk.word_tokenize(tweet)
    lemmatized_output = ' '.join([lemmatizer.lemmatize(w) for w in word_list])
    return lemmatized_output

In [30]:
df['tweet'] = df['tweet'].apply(lambda x : tweets_lemm(x))

In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords

In [32]:
nltk.download('punkt')
nltk.download('stopwords')
stops = set(stopwords.words('english'))

[nltk_data] Downloading package punkt to /home/rcrespillo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/rcrespillo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [33]:
vectorizer = TfidfVectorizer(analyzer='word', 
                             tokenizer=nltk.word_tokenize, 
                             stop_words=stops, 
                             ngram_range=(1,3))

X_train_v = vectorizer.fit(df['tweet'])

In [34]:
vectorizer.vocabulary_

{'father': 279465,
 'dysfunctional': 258865,
 'selfish': 512674,
 'drag': 255697,
 'kid': 366210,
 'dysfunction': 258862,
 '.': 109892,
 '#': 10299,
 'run': 500776,
 'father dysfunctional': 279784,
 'dysfunctional selfish': 258868,
 'selfish drag': 512685,
 'drag kid': 255708,
 'kid dysfunction': 366434,
 'dysfunction .': 258863,
 '. #': 109903,
 '# run': 62525,
 'father dysfunctional selfish': 279785,
 'dysfunctional selfish drag': 258869,
 'selfish drag kid': 512686,
 'drag kid dysfunction': 255709,
 'kid dysfunction .': 366435,
 'dysfunction . #': 258864,
 '. # run': 111942,
 'thanks': 557927,
 'lyft': 397457,
 'credit': 232513,
 'ca': 204761,
 "n't": 424547,
 'use': 587070,
 'cause': 210355,
 'offer': 444079,
 'wheelchair': 607695,
 'van': 588645,
 'pdx': 458202,
 'disapointed': 250332,
 'getthanked': 307230,
 'thanks #': 557936,
 '# lyft': 47841,
 'lyft credit': 397466,
 'credit ca': 232528,
 "ca n't": 204784,
 "n't use": 427275,
 'use cause': 587105,
 "cause n't": 210469,
 "n't o

In [35]:
combi[:31962].tail()

,id,label,tweet
31957,31958,0.0,ate isz that youuu ? ððððððð...
31958,31959,0.0,to see nina turner on the airwave trying to wr...
31959,31960,0.0,listening to sad song on a monday morning otw ...
31960,31961,1.0,"# sikh # temple vandalised in in # calgary , #..."
31961,31962,0.0,thank you for you follow


In [36]:
train.tail()

,id,label,tweet
31957,31958,0,ate @user isz that youuu?ðððððð...
31958,31959,0,to see nina turner on the airwaves trying to...
31959,31960,0,listening to sad songs on a monday morning otw...
31960,31961,1,"@user #sikh #temple vandalised in in #calgary,..."
31961,31962,0,thank you @user for you follow


In [38]:
df = combi[:31962]
df['label'] = df['label'].apply(int)
df_test = combi[31962:]

/home/rcrespillo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [39]:
df.tail()

,id,label,tweet
31957,31958,0,ate isz that youuu ? ððððððð...
31958,31959,0,to see nina turner on the airwave trying to wr...
31959,31960,0,listening to sad song on a monday morning otw ...
31960,31961,1,"# sikh # temple vandalised in in # calgary , #..."
31961,31962,0,thank you for you follow


In [40]:
X = df["tweet"]
y = df["label"]

In [41]:
from sklearn.model_selection import train_test_split

In [42]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, stratify=y)

In [43]:
X_train_v = vectorizer.transform(X_train)
X_test_v = vectorizer.transform(X_test)

In [44]:
X_train_v.shape

(25569, 645964)

In [45]:
from sklearn.neural_network import MLPClassifier

In [46]:
mlp_clf = MLPClassifier(hidden_layer_sizes=(10,), verbose=True)

In [47]:
mlp_clf.fit(X_train_v, y_train)

Iteration 1, loss = 0.68685951
Iteration 2, loss = 0.41884346
Iteration 3, loss = 0.23259868
Iteration 4, loss = 0.13948359
Iteration 5, loss = 0.08683146
Iteration 6, loss = 0.05649509
Iteration 7, loss = 0.03895366
Iteration 8, loss = 0.02846464
Iteration 9, loss = 0.02182293
Iteration 10, loss = 0.01743129
Iteration 11, loss = 0.01440387
Iteration 12, loss = 0.01225490
Iteration 13, loss = 0.01065687
Iteration 14, loss = 0.00944716
Iteration 15, loss = 0.00850328
Iteration 16, loss = 0.00775422
Iteration 17, loss = 0.00714698
Iteration 18, loss = 0.00664804
Iteration 19, loss = 0.00622627
Iteration 20, loss = 0.00586877
Iteration 21, loss = 0.00556081
Iteration 22, loss = 0.00529160
Iteration 23, loss = 0.00505504
Iteration 24, loss = 0.00484060
Iteration 25, loss = 0.00464634
Iteration 26, loss = 0.00446948
Iteration 27, loss = 0.00430510
Iteration 28, loss = 0.00415299
Iteration 29, loss = 0.00400984
Iteration 30, loss = 0.00387429
Iteration 31, loss = 0.00374532
Iteration 32, los

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(10,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=True, warm_start=False)

In [48]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score, classification_report

In [49]:
y_pred = mlp_clf.predict(X_test_v)

In [50]:
#confusion_matrix(y_test, y_pred)

In [51]:
print("roc_auc score: {}".format(roc_auc_score(y_test, y_pred)))
print(classification_report(y_test, y_pred))

roc_auc score: 0.7273421542712964
             precision    recall  f1-score   support

          0       0.96      1.00      0.98      5945
          1       0.98      0.46      0.62       448

avg / total       0.96      0.96      0.95      6393



In [52]:
X_all = vectorizer.transform(X)
y_all = y

In [53]:
mlp_clf.fit(X_all, y_all)

Iteration 1, loss = 0.37187057
Iteration 2, loss = 0.19711834
Iteration 3, loss = 0.11764722
Iteration 4, loss = 0.06719091
Iteration 5, loss = 0.03977318
Iteration 6, loss = 0.02577845
Iteration 7, loss = 0.01822890
Iteration 8, loss = 0.01384055
Iteration 9, loss = 0.01111182
Iteration 10, loss = 0.00930332
Iteration 11, loss = 0.00803352
Iteration 12, loss = 0.00711171
Iteration 13, loss = 0.00641456
Iteration 14, loss = 0.00586933
Iteration 15, loss = 0.00542660
Iteration 16, loss = 0.00505980
Iteration 17, loss = 0.00474739
Iteration 18, loss = 0.00447635
Iteration 19, loss = 0.00423642
Iteration 20, loss = 0.00401920
Iteration 21, loss = 0.00382265
Iteration 22, loss = 0.00364264
Iteration 23, loss = 0.00347678
Iteration 24, loss = 0.00332119
Iteration 25, loss = 0.00317655
Iteration 26, loss = 0.00304041
Iteration 27, loss = 0.00291314
Iteration 28, loss = 0.00279395
Iteration 29, loss = 0.00267699
Iteration 30, loss = 0.00257331
Iteration 31, loss = 0.00247243
Iteration 32, los

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(10,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=True, warm_start=False)

In [54]:
df_test.sample(5)

,id,label,tweet
37181,37182,NaN,# goodmorning # world it ` s a # rainingday an...
39938,39939,NaN,happy thursday ! do n't let the rain dampen yo...
48043,48044,NaN,soo excited to see you in chitty tonight ! eek...
47982,47983,NaN,"the fn is seeking to rid of it anti-semitic , ..."
43184,43185,NaN,my girl ð # familyman # mygirls # lovethem ...


In [55]:
dtest = vectorizer.transform(df_test['tweet'])
test_pred = mlp_clf.predict(dtest)

In [56]:
df_test['label'] = test_pred

/home/rcrespillo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [57]:
submission = df_test[['id','label']]

In [58]:
submission.to_csv('sub_rodixxi_mlpc_cleanData_lemma_3.csv', index=False)